### Install pyspark

In [24]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
# Import Apache Spark SQL
from pyspark.sql import SparkSession

# Create the Spark Session
spark = SparkSession \
    .builder \
    .appName("Streaming Process Files") \
    .config("spark.streaming.stopGracefullyOnShutdown", True) \
    .master("local[*]") \
    .getOrCreate()

spark

In [26]:
spark.sparkContext.setLogLevel("ERROR")

### Define the data structure

In [27]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([ \
    StructField("link", StringType()), \
    StructField("headline", StringType()), \
    StructField("category", StringType()), \
    StructField("short_description", StringType()), \
    StructField("authors", StringType()), \
    StructField("date", StringType()),         
])

## Determine the folder location

In [28]:
# create input-5025201258 folder
!mkdir -p input-5025201258

# upload datasets into files section then move it into input folder
!mv news-1.json input-5025201258/news-1.json
!mv news-2.json input-5025201258/news-2.json
!mv news-3.json input-5025201258/news-3.json

df = spark.readStream.schema(schema).json("input-5025201258/")

mv: cannot stat 'news-1.json': No such file or directory
mv: cannot stat 'news-2.json': No such file or directory
mv: cannot stat 'news-3.json': No such file or directory


In [29]:
df.printSchema()

root
 |-- link: string (nullable = true)
 |-- headline: string (nullable = true)
 |-- category: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- date: string (nullable = true)



## Do the aggregation step

In [30]:
groupDF = df.select("category").groupBy("category").count()
groupDF.printSchema()

root
 |-- category: string (nullable = true)
 |-- count: long (nullable = false)



## Start the streaming task

In [23]:
!mkdir -p output-5025201258

result = groupDF.writeStream \
  .format("console") \
  .option("path", "output-5025201258") \
  .outputMode("complete") \
  .start()


result.awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.9/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored